In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats as st
import scipy.linalg as la
from math import sqrt
from math import log
from math import exp
import math
import pandas as pd
from pandas import DataFrame
print("Modules Imported!")

Modules Imported!


# Team Indices

In [10]:
offense = pd.read_csv("OffensiveStats.csv", delimiter=',')
defense = pd.read_csv("DefensiveStats.csv", delimiter=',')
for index, row in offense.iterrows():
    print(index , row['Tm'])
    
def teamIndex(t):
    if t == "ARI":
        return 0
    elif t == "ATL":
        return 1
    elif t == "BAL":
        return 2
    elif t == "BUF":
        return 3
    elif t == "CAR":
        return 4
    elif t == "CHI":
        return 5
    elif t == "CIN":
        return 6
    elif t == "CLE":
        return 7
    elif t == "DAL":
        return 8
    elif t == "DEN":
        return 9
    elif t == "DET":
        return 10
    elif t == "GB":
        return 11
    elif t == "HOU":
        return 12
    elif t == "IND":
        return 13
    elif t == "JAX":
        return 14
    elif t == "KC":
        return 15
    elif t == "LV" or t == "OAK":
        return 16
    elif t == "LAC" or t== "SD":
        return 17
    elif t == "LA" or t== "STL":
        return 18
    elif t == "MIA":
        return 19
    elif t == "MIN":
        return 20
    elif t == "NE":
        return 21
    elif t == "NO":
        return 22
    elif t == "NYG":
        return 23
    elif t == "NYJ":
        return 24
    elif t == "PHI":
        return 25
    elif t == "PIT":
        return 26
    elif t == "SF":
        return 27
    elif t == "SEA":
        return 28
    elif t == "TB":
        return 29
    elif t == "TEN":
        return 30
    elif t == "WAS":
        return 31

0 Arizona Cardinals
1 Atlanta Falcons
2 Baltimore Ravens
3 Buffalo Bills
4 Carolina Panthers
5 Chicago Bears
6 Cincinnati Bengals
7 Cleveland Browns
8 Dallas Cowboys
9 Denver Broncos
10 Detroit Lions
11 Green Bay Packers
12 Houston Texans
13 Indianapolis Colts
14 Jacksonville Jaguars
15 Kansas City Chiefs
16 Las Vegas Raiders
17 Los Angeles Chargers
18 Los Angeles Rams
19 Miami Dolphins
20 Minnesota Vikings
21 New England Patriots
22 New Orleans Saints
23 New York Giants
24 New York Jets
25 Philadelphia Eagles
26 Pittsburgh Steelers
27 San Francisco 49ers
28 Seattle Seahawks
29 Tampa Bay Buccaneers
30 Tennessee Titans
31 Washington Football Team
32 Avg Team
33 League Total
34 Avg Tm/G


# Weights

In [11]:
#Needed Stats
OPass = offense["NY/A"].values
DPass = defense["NY/A"].values
ORush = offense["Y/A"].values
DRush = defense["Y/A"].values
OPen = offense["1stPy"].values
DPen = defense["1stPy"].values
OTO = offense["TO%"].values
DTO = defense["TO%"].values
OSc = offense["Sc%"].values
DSc = defense["Sc%"].values
OPly = offense["Ply"].values
DPly = defense["Ply"].values
OExp = offense["EXP"].values
DExp = defense["EXP"].values
Gs = offense["G"].values

teams = len(offense)
rows, cols = (teams, 15) 

def addStats(rows, cols):
    arr = np.zeros((rows, cols)) 
    arr[:,0] = OPass 
    arr[:,1] = DPass
    arr[:,2] = ORush
    arr[:,3] = DRush
    arr[:,4] = OPen
    arr[:,5] = DPen
    arr[:,6] = OTO
    arr[:,7] = DTO
    arr[:,8] = OSc 
    arr[:,9] = DSc
    arr[:,10] = OPly 
    arr[:,11] = DPly
    arr[:,12] = OExp
    arr[:,13] = DExp 
    arr[:,14] = Gs
    return arr
statArray = addStats(rows, cols)

In [12]:
def RunGame(team1, team2):
    x1 = (statArray[team1][0] + statArray[team2][1])*statArray[team1][10]/2
    y1 = (statArray[team2][0] + statArray[team1][1])*statArray[team2][10]/2
    x2 = (statArray[team1][2] + statArray[team2][3])*statArray[team1][10]/2
    y2 = (statArray[team2][2] + statArray[team1][3])*statArray[team2][10]/2
    x3 = (0.5*statArray[team1][4]/statArray[team1][14] + 0.5*statArray[team2][5]/statArray[team2][14])
    y3 = (0.5*statArray[team2][4]/statArray[team2][14] + 0.5*statArray[team1][5]/statArray[team1][14])
    x4 = (0.5*statArray[team1][6] + 0.5*statArray[team2][7])
    y4 = (0.5*statArray[team2][6] + 0.5*statArray[team1][7])
    x5 = (0.5*statArray[team1][8] + 0.5*statArray[team2][9])
    y5 = (0.5*statArray[team2][8] + 0.5*statArray[team1][9])
    x6 = (statArray[team1][10] + statArray[team2][11])/2
    y6 = (statArray[team2][10] + statArray[team1][11])/2
    home = 0.1
    return [x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,home]

#Must change this!!
RunGamesSize = 11

def CalculateGame(team1,team2,theta):
    X = RunGame(team1, team2)
    return X@theta

# Training

In [13]:
games = pd.read_csv("http://www.habitatring.com/games.csv")
games = games[games['season']>= 2020]
games = games[games['week']<=10]
games = games.filter(items = [ 'home_team','away_team', 'total','total_line'])

X = np.zeros((len(games),RunGamesSize))
Y = np.zeros(len(games))

i = 0
for index, row in games.iterrows():
    X[i] = RunGame(teamIndex(row['home_team']), teamIndex(row['away_team']))
    Y[i] = row['total']
    i+=1

Y = np.reshape(Y, [len(games),1])

xT = np.transpose(X)
yT = np.transpose(Y)

theta = la.inv(xT@X)@xT@Y
    
print(np.shape(X))
print(np.shape(Y))
print(np.shape(theta))
print(theta)

(147, 11)
(147, 1)
(11, 1)
[[ 5.81113747e-03]
 [ 2.22450255e-03]
 [-4.05636731e-04]
 [ 6.99945535e-03]
 [-1.73697283e+00]
 [ 3.60361658e+00]
 [-3.30778099e-01]
 [ 1.68644413e-01]
 [-2.73984338e-02]
 [ 4.55035004e-01]
 [-4.63533223e+02]]


# Testing Past Games

In [14]:
games = pd.read_csv("http://www.habitatring.com/games.csv")
games = games[games['season']==2020]
games = games[games['week']<=15]
games = games.filter(items = [ 'home_team','away_team', 'total','total_line', 'season'])

confidencePoints = 0
correct = 0;
total = 0;
for index, row in games.iterrows():
    predicted = float(CalculateGame(teamIndex(row['home_team']), teamIndex(row['away_team']),theta))
    predicted = round(predicted,1)
    spread = row['total_line']
    actual = row['total']
    if((predicted+confidencePoints<spread and actual<spread) or (predicted-confidencePoints>spread and actual>spread)):
        if(not math.isnan(actual)):
            #print(row['home_team'], "vs ", row['away_team'], row['season'])
            #print("Predicted Total: ", predicted)
            #print("Total Line: ", spread)
            #print("Actual Total: " , actual)
            correct+=1
            total+=1
    if(not math.isnan(actual) and abs(predicted-spread)>confidencePoints):
        if(not(predicted+confidencePoints<spread and actual<spread) and not(predicted-confidencePoints>spread and actual>spread)):
            #print(row['home_team'], "vs ", row['away_team'], row['season'])
            #print("Predicted Total: ", predicted)
            #print("Total Line: ", spread)
            #print("Actual Total: " , actual)
            total+=1
    
print("Predicted ", correct, "out of ", total, " games' totals correctly  with", confidencePoints, "points confidence")

Predicted  132 out of  223  games' totals correctly  with 0 points confidence


# Future Games

In [15]:
games = pd.read_csv("http://www.habitatring.com/games.csv")
games = games[games['season']==2020]
games = games[games['week']==17]
games = games.filter(items = [ 'home_team','away_team', 'total','total_line', 'season'])

total = 0

for index, row in games.iterrows():
    predicted = float(CalculateGame(teamIndex(row['home_team']), teamIndex(row['away_team']),theta))
    predicted = round(predicted,1)
    spread = row['total_line']
    if(abs(predicted-spread)>=confidencePoints):
        print(row['home_team'], "vs ", row['away_team'], row['season'])
        print("Predicted Total: ", predicted)
        print("Actual Total: ", spread)
        total+=1
        
print(total, "games with confidence of", confidencePoints, "points")

BUF vs  MIA 2020
Predicted Total:  51.6
Actual Total:  43.0
CIN vs  BAL 2020
Predicted Total:  49.2
Actual Total:  45.0
CLE vs  PIT 2020
Predicted Total:  44.5
Actual Total:  45.0
DET vs  MIN 2020
Predicted Total:  56.5
Actual Total:  54.0
NE vs  NYJ 2020
Predicted Total:  38.7
Actual Total:  41.0
NYG vs  DAL 2020
Predicted Total:  45.5
Actual Total:  43.5
TB vs  ATL 2020
Predicted Total:  48.0
Actual Total:  50.5
CAR vs  NO 2020
Predicted Total:  51.7
Actual Total:  46.5
CHI vs  GB 2020
Predicted Total:  49.3
Actual Total:  48.5
DEN vs  LV 2020
Predicted Total:  52.0
Actual Total:  50.0
HOU vs  TEN 2020
Predicted Total:  58.8
Actual Total:  54.0
IND vs  JAX 2020
Predicted Total:  49.9
Actual Total:  48.0
KC vs  LAC 2020
Predicted Total:  57.0
Actual Total:  43.5
LA vs  ARI 2020
Predicted Total:  49.4
Actual Total:  41.5
SF vs  SEA 2020
Predicted Total:  51.0
Actual Total:  46.0
PHI vs  WAS 2020
Predicted Total:  40.5
Actual Total:  43.0
16 games with confidence of 0 points


# Over/Unders

In [16]:
games = pd.read_csv("http://www.habitatring.com/games.csv")
games = games[games['season']<=2019]
games = games[games['season']>=2010]
games = games.filter(items = [ 'home_team','away_team', 'total','total_line', 'season'])

confidencePoints = 0
correct = 0;
total = 0;
for index, row in games.iterrows():
    predicted = float(CalculateGame(teamIndex(row['home_team']), teamIndex(row['away_team']),theta))
    predicted = round(predicted,1)
    spread = row['total_line']
    actual = row['total']
    if(spread>54):
            #print(row['home_team'], "vs ", row['away_team'], row['season'])
            #print("Predicted Total: ", predicted)
            #print("Total Line: ", spread)
            #print("Actual Total: " , actual)
        if(actual<spread):
            correct+=1
        total+=1
    
print("Predicted ", correct, "out of ", total, " games' totals correctly  with", confidencePoints, "points confidence")

Predicted  39 out of  70  games' totals correctly  with 0 points confidence
